In [ ]:
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Dense, Flatten, Multiply, Add, BatchNormalization, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Helper function to load and pad images
def load_and_pad_image(filepath, pad_width):
    image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    return np.pad(image, ((pad_width, pad_width), (pad_width, pad_width)), mode='constant')

# Function to get neighborhood
def get_neighbourhood(image_padded, i, j, pad_width):
    return image_padded[i - pad_width:i + pad_width + 1, j - pad_width:j + pad_width + 1].reshape(5, 5, 1)

# Function to prepare data for a given dataset
def prepare_data(image1, image2, ground_truth, pad_width):
    image1_padded = load_and_pad_image(image1, pad_width)
    image2_padded = load_and_pad_image(image2, pad_width)
    ground_truth = cv2.imread(ground_truth, cv2.IMREAD_GRAYSCALE)

    data = []
    labels = []
    for i in range(pad_width, image1_padded.shape[0] - pad_width):
        for j in range(pad_width, image1_padded.shape[1] - pad_width):
            neighbourhood1 = get_neighbourhood(image1_padded, i, j, pad_width)
            neighbourhood2 = get_neighbourhood(image2_padded, i, j, pad_width)
            data.append(np.dstack((neighbourhood1, neighbourhood2)))
            labels.append(1 if ground_truth[i - pad_width, j - pad_width] == 0 else 0)

    return np.array(data), np.array(labels), ground_truth

# Define attention gate
def attention_gate(x, g, inter_channels):
    theta_x = Conv2D(inter_channels, kernel_size=1, strides=1, padding='same')(x)
    phi_g = Conv2D(inter_channels, kernel_size=1, strides=1, padding='same')(g)
    add_xg = Add()([theta_x, phi_g])
    relu_xg = tf.keras.layers.Activation('relu')(add_xg)
    psi = Conv2D(1, kernel_size=1, strides=1, padding='same', activation='sigmoid')(relu_xg)
    return Multiply()([x, psi])

# Define U-Net with attention model
def unet_with_attention(input_shape=(5, 5, 2)):
    inputs = Input(input_shape)

    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Dropout(0.2)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D((1, 1))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D((1, 1))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)

    aspp_1 = Conv2D(256, (3, 3), dilation_rate=1, activation='relu', padding='same')(conv4)
    aspp_2 = Conv2D(256, (3, 3), dilation_rate=2, activation='relu', padding='same')(conv4)
    aspp_3 = Conv2D(256, (3, 3), dilation_rate=4, activation='relu', padding='same')(conv4)
    aspp_4 = Conv2D(256, (3, 3), dilation_rate=8, activation='relu', padding='same')(conv4)
    aspp_out = Concatenate()([aspp_1, aspp_2, aspp_3, aspp_4])
    aspp_out = Conv2D(256, (1, 1), activation='relu', padding='same')(aspp_out)

    up5 = UpSampling2D((1, 1))(aspp_out)
    att5 = attention_gate(conv3, up5, inter_channels=128)
    concat5 = Concatenate()([up5, att5])
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(concat5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)

    up6 = UpSampling2D((1, 1))(conv5)
    att6 = attention_gate(conv2, up6, inter_channels=64)
    concat6 = Concatenate()([up6, att6])
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(concat6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)

    up7 = UpSampling2D((1, 1))(conv6)
    att7 = attention_gate(conv1, up7, inter_channels=32)
    concat7 = Concatenate()([up7, att7])
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(concat7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)

    output = Flatten()(conv7)
    output = Dense(1, activation='sigmoid')(output)
    model = Model(inputs, output)
    return model

# Define datasets
datasets = [
    ("/content/set11.tif", "/content/set12.tif", "/content/gt1.tif"),
    ("/content/set21.tif", "/content/set22.tif", "/content/gt2.tif"),
    ("/content/set31.tif", "/content/set32.tif", "/content/gt3.tif"),
    ("/content/set41.tif", "/content/set42.tif", "/content/gt4.tif"),
    ("/content/set51.tif", "/content/set52.tif", "/content/gt5.tif"),
    ("/content/set61.tif", "/content/set62.tif", "/content/gt6.tif")
]

# Split datasets into training and testing sets
train_datasets = datasets[:3]
test_datasets = datasets[3:]

# Prepare training data
pad_width = 2
X_train, y_train = [], []

for img1_path, img2_path, gt_path in train_datasets:
    data, labels, _ = prepare_data(img1_path, img2_path, gt_path, pad_width)
    X_train.append(data)
    y_train.append(labels)

X_train = np.concatenate(X_train)
y_train = np.concatenate(y_train)

# Prepare testing data
X_test, y_test, ground_truths = [], [], []

for img1_path, img2_path, gt_path in test_datasets:
    data, labels, ground_truth = prepare_data(img1_path, img2_path, gt_path, pad_width)
    X_test.append(data)
    y_test.append(labels)
    ground_truths.append(ground_truth)

X_test = np.concatenate(X_test)
y_test = np.concatenate(y_test)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)

# Build and train the model
model = unet_with_attention(input_shape=(5, 5, 2))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
lr_scheduler = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=16,
    class_weight=class_weights,
    callbacks=[lr_scheduler, early_stopping],
    verbose=1
)

# Predict and evaluate
predictions = model.predict(X_test)
predicted_maps = (predictions > 0.5).astype(int)

print("Classification Report:")
y_pred = predicted_maps.flatten()
print(classification_report(y_test, y_pred))

# Visualization
plt.figure(figsize=(18, 12))
for idx, gt in enumerate(ground_truths):
    plt.subplot(len(ground_truths), 2, 2 * idx + 1)
    plt.imshow(gt, cmap='gray')
    plt.title(f"Ground Truth {idx + 4}")
    plt.axis("off")

    predicted_map = predicted_maps[idx * gt.size:(idx + 1) * gt.size].reshape(gt.shape)
    plt.subplot(len(ground_truths), 2, 2 * idx + 2)
    plt.imshow(predicted_map, cmap='gray')
    plt.title(f"Predicted Change Map {idx + 4}")
    plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score

# Function to calculate PCC, PFA, and other metrics
def calculate_metrics(y_true, y_pred):
    # Calculate confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    if cm.size == 4:  # Ensure it's a binary classification confusion matrix
        TN, FP, FN, TP = cm.ravel()

        # Invert the confusion matrix for opposite labels
        # Since predicted and ground truth are opposite
        TN, FP, FN, TP = FP, TN, TP, FN

        # Calculate metrics
        N0 = TP + FN  # Total number of actual "change" pixels
        N1 = TN + FP  # Total number of actual "no-change" pixels

        # Probabilistic Conditional Correctness (PCC)
        PCC = ((TP + TN) / (N0 + N1)) * 100 if (N0 + N1) != 0 else 0

        # Probability of False Alarm (PFA)
        PFA = FP / N1 if N1 != 0 else 0

        # Total Error (TE)
        PTE = (FN + FP) / (N0 + N1) if (N0 + N1) != 0 else 0

        return PCC, PFA, PTE
    else:
        raise ValueError("Confusion matrix size is not 4. Ensure binary classification.")

# Initialize lists for storing results
accuracies = []
y_true_list = []
y_pred_list = []

# Flatten predictions and ground truths for the test set
for idx, ground_truth in enumerate(ground_truths):
    gt_flat = ground_truth.flatten()
    predicted_map_flat = predicted_maps[idx * ground_truth.size:(idx + 1) * ground_truth.size].flatten()

    # Ensure binary classification
    y_true = np.where(gt_flat > 0.5, 1, 0)  # Convert to binary
    y_pred = np.where(predicted_map_flat > 0.5, 1, 0)  # Convert to binary

    # Calculate PCC, PFA, and PTE
    PCC, PFA, PTE = calculate_metrics(y_true, y_pred)

    # Calculate accuracy for the fold
    accuracy = accuracy_score(y_true, y_pred)
    accuracies.append(accuracy)
    y_true_list.append(y_true)
    y_pred_list.append(y_pred)
    accuracy = 1-accuracy

    # Print per dataset metrics
    print(f"Dataset {idx + 4} Metrics:")
    print(f"PCC: {PCC}%")
    print(f"PFA: {PFA}")
    print(f"PTE: {PTE}")
    print(f"Accuracy: {accuracy * 100}%")
    print("-" * 50)

# Print overall performance across all datasets
all_true = np.concatenate(y_true_list)
all_pred = np.concatenate(y_pred_list)

# Calculate confusion matrix and metrics for all datasets
cm_all = confusion_matrix(all_true, all_pred)
TN_all, FP_all, FN_all, TP_all = cm_all.ravel()

N0_all = TP_all + FN_all
N1_all = TN_all + FP_all

# Probabilistic Conditional Correctness (PCC)
PCC_all = ((TP_all + TN_all) / (N0_all + N1_all)) * 100 if (N0_all + N1_all) != 0 else 0

# Probability of False Alarm (PFA)
PFA_all = FP_all / N1_all if N1_all != 0 else 0

# Probability of Total Error (PTE)
PTE_all = (FN_all + FP_all) / (N0_all + N1_all) if (N0_all + N1_all) != 0 else 0

# Print final performance
print(f'Final Accuracy (across all datasets): {(1-np.mean(accuracies)) * 100}%')
print(f'Confusion Matrix (all datasets):\n{cm_all}')
print(f'Correct Classification (CC) across all datasets: {sum(all_true == all_pred)}')
print(f'False Alarms (FA) across all datasets: {sum(all_pred == 1) - sum(all_true == 1)}')
print(f'Total Error (TE) across all datasets: {sum(all_true != all_pred)}')
print(f'Probabilistic Conditional Correctness (PCC) across all datasets: {PCC_all}%')
print(f'Probability of False Alarm (PFA) across all datasets: {PFA_all}')
print(f'Probability of Total Error (PTE) across all datasets: {PTE_all}')

# Visualization of ground truth and predicted maps for each dataset
plt.figure(figsize=(18, 12))
for idx, ground_truth in enumerate(ground_truths):
    # Plot Ground Truth
    plt.subplot(len(ground_truths), 2, 2 * idx + 1)
    plt.imshow(ground_truth, cmap='gray')
    plt.title(f"Ground Truth {idx + 4}")
    plt.axis("off")

    # Plot Predicted Map
    predicted_map = predicted_maps[idx * ground_truth.size:(idx + 1) * ground_truth.size].reshape(ground_truth.shape)

    # Invert the predicted map to show the opposite
    inverted_predicted_map = 1 - predicted_map

    plt.subplot(len(ground_truths), 2, 2 * idx + 2)
    plt.imshow(inverted_predicted_map, cmap='gray')
    plt.title(f"Inverted Predicted Change Map {idx + 4}")
    plt.axis("off")

plt.tight_layout()
plt.show()
